# Bài 3 - Sửa lỗi lượng tử

## Bối cảnh lịch sử

Thuật toán Shor đã cho máy tính lượng tử đã mang lại cho máy tính lượng tử một trường hợp sử dụng đáng giá - nhưng sự nhiễu vốn có của cơ học lượng tử đồng nghĩa với việc xây dựng nên phần cứng có khả năng chạy một thuật toán như vậy sẽ rất khó. Năm 1995, Shor phát hành một bài báo mang tính bước ngoặt khác: một chương tình chia sẻ thông tin lượng tử qua nhiều qubit để giảm lỗi. [1] 

Rất nhiều tiến bộ đã được thực hiện trong nhiều thập kỷ kể từ đó. Các dạng mã sửa lỗi mới đã được phát hiện và một khuôn khổ thuyết lớn đã được xây dựng xung quanh chúng. Các mã bề mặt (surface codes) do Kitaev đề xuất vào năm 1997 đã nổi lên như một ứng cử viên hàng đầu, và nhiều biến thể trên thiết kế ban đầu đã xuất hiện kể từ đó. Nhưng vẫn còn rất nhiều sự tiến bộ cần đạt được trong việc điều chỉnh mã cho phù hợp với các chi tiết cụ thể của phần cứng lượng tử. [2] 

Trong bài tập này, chúng ta sẽ xem xét một trường hợp trong đó các 'lỗi' nhân tạo được chèn vào mạch lượng tử. Nhiệm vụ của bạn là thiết kế mạch sao cho có thể xác định được các cổng bổ sung. 

Sau đó, bạn sẽ cần phải suy nghĩ về cách triển khai mạch lượng tử của mình trên một thiết bị thực. Điều này có nghĩa là bạn sẽ cần điều chỉnh giải pháp của mình cho phù hợp với bố cục của các qubit. Lời giải của bạn sẽ được tính điểm dựa trên số lượng cổng rối lượng tử (loại cổng nhiễu nhất) mà bạn sử dụng. 

### Tài liệu tham khảo
1. Shor, Peter W. "Scheme for reducing decoherence in quantum computer memory." Physical review A 52.4 (1995): R2493.
1. Dennis, Eric, et al. "Topological quantum memory." Journal of Mathematical Physics 43.9 (2002): 4452-4505.

## Bài tập về lỗi

Lỗi xảy ra khi một số hoạt động giả mạo tác động lên qubit của chúng ta. Tác động của chúng khiến mọi thứ hoạt động trở nên sai trong mạch của chúng ta. Các kết quả kỳ lạ mà bạn thấy khi chạy trên các thiết bị thực đều là do những lỗi này. 

Có rất nhiều tác động giả có thể xảy ra, nhưng hóa ra chúng ta có thể giả vờ rằng chỉ có hai loại lỗi: lệch bit và lệch pha. 

Lệch bit có tác dụng tương tự như cổng `x`. Nó chuyển trạng thái $|0\rangle$ của một qubit thành $|1\rangle$ và ngược lại. Lệch pha có tác dụng tương tự như cổng `z`, đem pha $-1$ vào trạng thái chồng chập. Nói một cách đơn giản, nó chuyển trạng thái $|+\rangle$ của một qubit thành $|-\rangle$ và ngược lại.

Lý do chúng ta có thể nghĩ rằng bất kỳ lỗi nào cũng chỉ trong hai lỗi này là bởi vì bất kỳ lỗi nào cũng có thể được biểu diễn bằng một ma trận và bất kỳ ma trận nào cũng có thể được viết dưới dạng ma trận $X$ và $Z$. Cụ thể, đối với bất kỳ ma trận qubit nào $M$, 

$$
M = \alpha I + \beta X + \gamma XZ + \delta Z,
$$

với một số giá trị phù hợp được chọn $\alpha$, $\beta$, $\gamma$ and $\delta$.

Vì vậy, bất cứ khi nào chúng ta áp dụng ma trận này cho một trạng thái qubit đơn lẻ  $|\psi\rangle$ chúng ta sẽ nhận được

$$
M |\psi\rangle = \alpha |\psi\rangle + \beta X |\psi\rangle + \gamma XZ |\psi\rangle + \delta Z |\psi\rangle.
$$

Kết quả cho ra trạng thái chồng chất bao gồm trạng thái ban đầu, trạng thái mà chúng ta sẽ có nếu lỗi là lỗi lệch bit, trạng thái cho lỗi lệch pha và trạng thái cho cả hai lỗi. Nếu chúng ta có một số cách để đo lường xem lỗi lệch pha hay lệch bit đã xảy ra, thì trạng thái sau đó sẽ sụp đổ về chỉ còn một khả năng. Và lỗi phức tạp của chúng ta sẽ trở thành lỗi lệch bit hoặc lỗi lệch pha đơn giản. 

Vậy làm thế nào để chúng ta phát hiện ra mình bị lỗi lệch bit hay lệch pha (hoặc cả hai). Và chúng ta sẽ làm gì với nó khi chúng ta đã biết? Trả lời những câu hỏi này chính là việc sửa lỗi lượng tử. 




## Một ví dụ rất đơn giản 

Một trong những mạch lượng tử đầu tiên mà hầu hết mọi người từng viết là tạo ra một cặp qubit rối với nhau. Trong hành trình sửa lỗi lượng tử này, chúng ta sẽ bắt đầu theo cùng cách này. 

In [ ]:
from qiskit import QuantumCircuit, Aer

# Tạo một cặp rối lượng tử
qc_init = QuantumCircuit(2)
qc_init.h(0)
qc_init.cx(0,1)

# vẽ mạch
display(qc_init.draw('mpl'))

# Nhận kết quả
qc = qc_init.copy()
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Ở đây, chúng ta thấy kết quả mong đợi khi chúng ta chạy mạch: trạng thái  `00` và `11` xảy ra với xác suất ngang nhau.

Nhưng điều gì sẽ xảy ra khi chúng ta dùng cùng một mạch, nhưng với một 'lỗi' lệch bit được chèn bằng tay. 

In [ ]:
# Tạo ra một lỗi lệch bit
qc_insert = QuantumCircuit(2)
qc_insert.x(0)

# Thêm vào nó mạch gốc
qc = qc_init.copy()
qc = qc.compose(qc_insert)

# Vẽ mạch
display(qc.draw('mpl'))

# Nhận kết quả
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Bây giờ kết qua đã thay đổi: `01` và `10`. Hai giá trị bit đã đi từ luôn đồng ý sang luôn không đồng ý. Bằng cách này, chúng tôi phát hiện ra ảnh hưởng của lỗi. 

Một cách khác mà chúng ta có thể phát hiện lỗi là gỡ bỏ rối lượng tử bằng một vài cổng nữa. Nếu không có lỗi, chúng ta sẽ quay lại trạng thái ban đầu $|00\rangle$.

In [ ]:
# Gỡ rối lượng tử
qc_syn = QuantumCircuit(2)
qc_syn.cx(0,1)
qc_syn.h(0)

# Thêm nó vào sau lỗi
qc = qc_init.copy()
qc = qc.compose(qc_syn)

# Vẽ mạch
display(qc.draw('mpl'))

# Nhận kết quả
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Nhưng điều gì sẽ xảy ra nếu có lỗi ở một trong các qubit? Hãy thử chèn các lỗi khác nhau để tìm hiểu. 

Đây là một mạch với tất cả các thành phần mà chúng tôi đã giới thiệu cho đến nay: phần khởi tạo `qc_init`, phần thêm lỗi `qc_insert` và cuối cùng phần `qc_syn` điều này đảm bảo rằng phép đo cuối cùng cho một câu trả lời xác đáng. 

In [ ]:
# Tạo ra một lỗi
qc_insert = QuantumCircuit(2)
qc_insert.x(0)

# Gỡ rối lượng tử
qc_syn = QuantumCircuit(2)
qc_syn.cx(0,1)
qc_syn.h(0)

# Thêm nó vào sau lỗi
qc = qc_init.copy()
qc = qc.compose(qc_insert)
qc = qc.compose(qc_syn)

# Vẽ mạch
display(qc.draw('mpl'))

# Nhận kết quả
qc.measure_all()
job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Bạn sẽ thấy rằng đầu ra cho chúng ta biết chính xác những gì đang xảy ra với các lỗi. Cả lỗi lệch pha và lỗi lệch bit đều có thể được xác định. Giá trị bit bên trái là `1` chỉ khi xảy ra lỗi lệch bit (nếu chúng tôi đã chèn một `x(0)` hoặc `x(1)`). Tương tự như vậy bit bên phải cho chúng ta biết nếu có xảy ra lỗi lệch pha hay không (chèn một `z(0)` hoặc `z(1)`).

Khả năng phát hiện và phân biệt lệch bit và lệch pha này rất hữu ích. Nhưng nó chưa đủ hữu ích. Chúng tôi chỉ có thể cho biết *loại lỗi* nào đang xảy ra, chứ không thể biết *ở đâu*. Nếu không có thêm chi tiết thì không thể tìm ra cách loại bỏ ảnh hưởng của các hoạt động này khỏi các tính toán của chúng ta. Do đó, để sửa lỗi lượng tử, chúng ta cần một thứ gì đó lớn hơn và tốt hơn.

Nhiệm vụ của bạn là làm điều đó! Đây là danh sách những gì bạn cần gửi. Mọi thứ ở đây sẽ được giải thích sau bằng ví dụ theo kèm. 

<div class="alert alert-block alert-success">

<b>Mục tiêu</b>

Tạo một mạch có thể phát hiện lỗi `x` và `z` trên hai qubits.
Bạn có thể đưa ra một giải pháp của riêng bạn. Hoặc chỉ cần điều chỉnh giải pháp gần như hợp lệ được đưa ra bên dưới. 
    
</div>

<div class="alert alert-block alert-danger">
<b>Những thứ để nộp</b> 

* Bạn cần cung cấp hai mạch lượng tử:
    * `qc_init`: Chuẩn bị các qubits (trong đó có ít nhất hai ) ở trạng thái mong muốn ban đầu;
    * `qc_syn`: Đo lường một tập hợp con của các qubit.

* Các lỗi giả được chèn là cổng `x` và` z` trên hai qubit cụ thể. Bạn cần chọn hai qubit để sử dụng cho việc này (được cung cấp dưới dạng danh sách `error_qubits`). 

* Có 16 tập hợp lỗi có thể được chèn vào (bao gồm cả trường hợp nhỏ không có lỗi). Kết quả đo của `qc_syn` sẽ xuất ra một chuỗi bit duy nhất cho mỗi tập hợp. Bộ chấm điểm sẽ trả về thông báo lỗi *'Please make sure the circuit is created to the initial layout. (Hãy đảm bảo rằng mạch được tạo theo bố cục ban đầu.)'* Nếu điều này không được thỏa mãn. 

* Bộ chấm điểm sẽ biên dịch mạch hoàn chỉnh cho `ibmq_tokyo` (một thiết bị đã ngừng hoạt động). Để thấy rằng giải pháp của bạn được thiết kế riêng cho thiết bị, sự chuyển đổi này sẽ không thay đổi số lượng cổng `cx`. Nếu nó thay đổi, bạn sẽ nhận được thông báo lỗi *'Hãy đảm bảo rằng mạch được tạo theo bố cục ban đầu.'* 
    
* Để hướng dẫn chuyển đổi, bạn sẽ cần phải cho trình chuyển đổi biết qubit nào trên thiết bị sẽ được sử dụng như qubit nào trong mạch của bạn. Điều này được thực hiện với danh sách `initial_layout`. 
    
* Bạn có thể bắt đầu với ví dụ được đưa ra dưới đây, ví dụ này có thể trở thành câu trả lời hợp lệ chỉ với một vài chỉnh sửa. 
</div>

## Một ví dụ tốt hơn: mã bề mặt (surface code)

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile

import qiskit.tools.jupyter
from qiskit.test.mock import FakeTokyo

Trong ví dụ này, chúng ta sẽ sử dụng 5 qubit mà chúng ta sẽ gọi là mã qubit (code qubit). Để theo dõi chúng, chúng tôi sẽ sử dụng một thanh ghi lượng tử đặc biệt. 

In [ ]:
code = QuantumRegister(5,'code')

Chúng tôi cũng sẽ có thêm bốn qubit mà chúng tôi gọi là qubit syndrome. 

In [ ]:
syn = QuantumRegister(4,'syn')

Tương tự, chúng tôi xác định một thanh ghi cho bốn bit đầu ra, được sử dụng khi đo các qubit syndrome. 

In [ ]:
out = ClassicalRegister(4,'output')

Chúng tôi xem xét các qubit được sắp xếp như sau, với các qubit mã tạo thành các góc của bốn hình tam giác, và các qubit syndrome ở bên trong mỗi tam giác. 

```
c0----------c1
| \   s0   / |
|   \    /   |
| s1  c2  s2 |
|   /    \   |
| /   s3   \ |
c3----------c4
```

Đối với mỗi tam giác, chúng tôi liên kết hoạt động của bộ ổn định trên ba qubit của nó. Đối với qubit ở hai bên, bộ ổn định là ZZZ. Đối với qubit trên cùng và dưới cùng, chúng là XXX. 

Mạch đo syndrome tương ứng với phép đo các vật quan sát (observables) này. Điều này được thực hiện theo cách tương tự với bộ ổn định mã bề mặt (trên thực tế, đoạn code này là một phiên bản nhỏ của mã bề mặt). 
<div class="alert alert-block alert-danger">
 
<b>Cảnh báo</b> 

Bạn nên loại bỏ các thanh chắn(barriers) trước khi gửi mã vì nó có thể cản trở quá trình chuyển đổi. Nó được để ở đây để dễ hình dung. </div>

In [ ]:
qc_syn = QuantumCircuit(code,syn,out)


# ZZZ trái
qc_syn.cx(code[0],syn[1])
qc_syn.cx(code[2],syn[1])
qc_syn.cx(code[3],syn[1])
qc_syn.barrier()

# ZZZ phải
qc_syn.cx(code[1],syn[2])
qc_syn.cx(code[2],syn[2])
qc_syn.cx(code[4],syn[2])
qc_syn.barrier()

# XXX trên
qc_syn.h(syn[0])
qc_syn.cx(syn[0],code[0])
qc_syn.cx(syn[0],code[1])
qc_syn.cx(syn[0],code[2])
qc_syn.h(syn[0])
qc_syn.barrier()

# XXX dưới
qc_syn.h(syn[3])
qc_syn.cx(syn[3],code[2])
qc_syn.cx(syn[3],code[3])
qc_syn.cx(syn[3],code[4])
qc_syn.h(syn[3])
qc_syn.barrier()


# Đo các qubit phụ trợ (auxiliary qubit)
qc_syn.measure(syn,out)
qc_syn.draw('mpl')

Mạch khởi tạo chuẩn bị một trạng thái riêng(eigenstate) của các vật thể quan sát (observables) này, để cho chắc chắn rằng đầu ra của phép đo syndrome sẽ là `0000`. 

In [ ]:
qc_init = QuantumCircuit(code,syn,out)

qc_init.h(syn[0])
qc_init.cx(syn[0],code[0])
qc_init.cx(syn[0],code[1])
qc_init.cx(syn[0],code[2])
qc_init.cx(code[2],syn[0])

qc_init.h(syn[3])
qc_init.cx(syn[3],code[2])
qc_init.cx(syn[3],code[3])
qc_init.cx(syn[3],code[4])
qc_init.cx(code[4],syn[3])

qc_init.barrier()
qc_init.draw('mpl')

Hãy kiểm tra xem điều đó có đúng không. 

In [ ]:
qc = qc_init.compose(qc_syn)
display(qc.draw('mpl'))

job = Aer.get_backend('qasm_simulator').run(qc)
job.result().get_counts()

Bây giờ chúng ta hãy tạo một mạch mà chúng ta có thể chèn các cổng `x` và` z` trên hai qubit của chúng ta. Để làm điều này, chúng ta sẽ cần chọn trong  số 5 qubit mà chúng ta có sẽ tương ứng với hai qubit được yêu cầu cho điều kiện hợp lệ.

Đối với đoạn code này, chúng ta cần chọn những góc đối diện. 

In [ ]:
error_qubits = [0,4]

Ở đây 0 và 4 đề cập đến vị trí của các qubit trong danh sách sau, và do đó là các qubit `code[0]` và `code[4]`. 

In [ ]:
qc.qubits

Để kiểm tra xem đoạn code có thực hiện như chúng ta yêu cầu hay không, chúng ta có thể sử dụng hàm sau để tạo các mạch để chèn lỗi nhân tạo. Ở đây, các lỗi mà chúng tôi muốn thêm được liệt kê trong `error` dưới dạng một chuỗi văn bản đơn giản, chẳng hạn như` x0` tượng trưng cho một cổng `x` trên` error_qubits [0] `. 

In [ ]:
def insert(errors,error_qubits,code,syn,out):

    qc_insert = QuantumCircuit(code,syn,out)

    if 'x0' in errors:
        qc_insert.x(error_qubits[0])
    if 'x1' in errors:
        qc_insert.x(error_qubits[1])
    if 'z0' in errors:
        qc_insert.z(error_qubits[0])
    if 'z1' in errors:
        qc_insert.z(error_qubits[1])
        
    return qc_insert

Thay vì tất cả 16 khả năng, chúng ta hãy chỉ xem xét bốn trường hợp mà một lỗi duy nhất được chèn vào.

In [ ]:
for error in ['x0','x1','z0','z1']:
    
    qc = qc_init.compose(insert([error],error_qubits,code,syn,out)).compose(qc_syn)
    job = Aer.get_backend('qasm_simulator').run(qc)
    
    print('\nFor error '+error+':')
    counts = job.result().get_counts()
    for output in counts:
        print('Output was',output,'for',counts[output],'shots.')

Ở đây chúng ta thấy rằng mỗi bit trong đầu ra là `1` khi một lỗi cụ thể xảy ra: phía ngoài cùng bên trái phát hiện` z` trên `error_qubits [1]`, sau đó phía tiếp theo phát hiện `x` trên` error_qubits [1] `, và tiếp tục như vậy. 

<div class="alert alert-block alert-danger">
 
<b>Chú ý</b> 

Thứ tự chính xác của đầu ra rất quan trọng đối với bài tập này. Vui lòng làm theo thứ tự như dưới đây: 
1. Đầu ra của phía ngoài cùng bên trái đại diện cho  `z` trên `code[1]`.
2. Đầu ra thứ hai từ bên trái đại diện cho  `x` trên `code[1]`.
3. Đầu ra thứ ba từ bên trái đại diện cho `x` on `code[0]`.
4. Đầu ra của phía ngoài cùng bên phải đại diện cho `z` trên `code[0]`.
    
</div>

Khi có nhiều lỗi hơn ảnh hưởng đến mạch, ta khó có thể phân biệt rõ ràng lỗi nào đã xảy ra. Tuy nhiên, bằng cách liên tục lặp lại quá trình đọc đầu ra của syndrome để có thêm kết quả và phân tích dữ liệu thông qua quá trình giải mã, ta vẫn có thể xác định đủ về các lỗi và điều chỉnh tác động của chúng. 

Những cân nhắc như vậy nằm ngoài những gì chúng ta sẽ xem xét trong thử thách này. Thay vào đó, chúng ta sẽ tập trung vào một cái gì đó đơn giản hơn, nhưng cũng quan trọng không kém: bạn càng có ít lỗi và chúng càng đơn giản, thì việc sửa lỗi của bạn phải càng tốt. Để đảm bảo điều này, quy trình sửa lỗi của bạn nên được thực hiện phù hợp với thiết bị bạn đang sử dụng. 

Trong thử thách này, chúng tôi sẽ xem xét thiết bị `ibmq_tokyo`. Mặc dù phiên bản thực của thiết bị này đã bị gỡ bỏ cách đây một thời gian, nhưng nó vẫn tồn tại như một trong những thiết bị giả lập. 

In [ ]:
# Hãy dùng backend được cho sẵn ở đây
backend = FakeTokyo()
backend

Như một ý tưởng đơn giản về việc mạch gốc của chúng ta được trình bày ra sao, hãy xem nó chứa bao nhiêu cổng hai qubit. 

In [ ]:
qc = qc_init.compose(qc_syn)
qc = transpile(qc, basis_gates=['u','cx'])
qc.num_nonlocal_gates()

Nếu chúng tôi chuyển đổi nó sang thiết bị `ibmq_tokyo`, việc ánh xạ lại sẽ cần phải xảy ra với chi phí là việc thêm vào các cổng hai qubit. 

In [ ]:
qc1 = transpile(qc,backend,basis_gates=['u','cx'], optimization_level=3)
qc1.num_nonlocal_gates()

Chúng tôi có thể kiểm soát điều này ở một mức độ nào đó bằng cách xem những qubit nào trên thiết bị sẽ có thể được sử dụng tốt nhất làm những qubit trong code của ta. Nếu chúng ta xem xét những qubit nào trong đoạn code cần được kết nối bằng cổng hai qubit trong `qc_syn`, chúng ta tìm thấy biểu đồ kết nối bắt buộc sau đây. 

```
c0....s0....c1
:      :     :        
:      :     :
s1....c2....s2
:      :     :
:      :     :
c3....s3....c4
```

Không có tập hợp qubit nào trên `ibmq_tokyo` có thể cung cấp điều này, nhưng một số tập hợp nhất định như 0,1,2,5,6,7,10,11,12 sẽ khá giống. Vì vậy, chúng ta có thể thiết lập một `Initial_layout` để yêu cầu trình chuyển đổi sử dụng chúng. 



In [ ]:
initial_layout = [0,2,6,10,12,1,5,7,11]

Chúng cho bộ chuyển tiếp biết qubit nào trên thiết bị sẽ được sử dụng cho các qubit trong mạch (thứ tự chúng được liệt kê trong `qc.qubits`). Vì vậy, năm giá trị đầu tiên trong danh sách này cho mạch biết qubit nào sẽ sử dụng làm code qubit(mã qubit) và bốn mục tiếp theo trong danh sách này cũng tương tự đối với các qubit syndorme. Vì vậy, chúng tôi sử dụng qubit 0 trên thiết bị làm `code[0]`, qubit 2 là `code[1]`và tiếp tục như vậy.

Bây giờ chúng ta hãy sử dụng nó để chuyển đổi. 

In [ ]:
qc2 = transpile(qc,backend,initial_layout=initial_layout, basis_gates=['u','cx'], optimization_level=3)
qc2.num_nonlocal_gates()

Mặc dù quá trình chuyển mã là một quá trình ngẫu nhiên, bạn sẽ thường thấy rằng quá trình này sử dụng ít cổng hai qubit hơn so với khi không có bố cục ban đầu(intial layout) nào được cung cấp (bạn có thể chạy lại cả hai code chuyển dịch nhiều lần để xem quá trình chuyển mã là một quá trình ngẫu nhiên).

Tuy nhiên, một sơ đồ sửa lỗi được thiết kế đúng cách sẽ không cần bất kỳ việc ánh xạ lại nào. Nó phải được viết chính xác cho thiết bị được sử dụng và số lượng cổng hai qubit phải chắc chắn không đổi. Đây là điều kiện để lời giải được công nhận. Vì vậy, bạn sẽ không chỉ cung cấp một `Initial_layout`, mà còn phải thiết kế mạch của bạn riêng cho bố cục đó.

Nhưng phần đó chúng tôi để lại cho bạn! 

In [ ]:
# Kiểm tra đáp án của bạn sử dụng code sau
from qc_grader import grade_ex3
grade_ex3(qc_init,qc_syn,error_qubits,initial_layout)

In [ ]:
# Nộp kết quả của bạn. Bạn có thể nộp lại kết quả bất kì lúc nào.
from qc_grader import submit_ex3
submit_ex3(qc_init,qc_syn,error_qubits,initial_layout)

## Thông tin thêm

**Tạo bởi:** James Wootton, Rahul Pratap Singh

**Dịch bởi:** Bao Bach

**Phiên bản:** 1.0.1